# Homework nr. 2 - data cleaning (deadline 8/11/2018)

In short, the main task is to clean The Metropolitan Museum of Art Open Access dataset.
  
> The instructions are not given in details: It is up to you to come up with ideas on how to fulfill the particular tasks as best you can. ;)

## What are you supposed to do:

  1. Download the dataset MetObjects.csv from the repository https://github.com/metmuseum/openaccess/.
  2. Check consistency of at least three features where you expect problems (include "Object Name" feature).
  3. Select some features where you expect integrity problems (describe your choice) and check integrity of those features.
  4. Convert at least five features to a proper data type. Choose at least one numeric, one categorical and one datetime.
  5. Find some outliers (describe your selection criteria).
  6. Detect missing data in at least three features, convert them to a proper representation (if they are already not), and impute missing values in at least one feature.

**If you do all this properly, you will obtain 6 points**

To earn **extra two points** you can do some of these:
  * Focus more precisely on cleaning of the "Medium" feature. Such if you like to use it in KNN based algorithms later.
  * Focus on the extraction of physical dimensions of each item (width, depth and height in centimeters) from the "Dimensions" feature.

## Comments

  * Please follow the instructions from https://courses.fit.cvut.cz/MI-PDD/homeworks/index.html.
  * If the reviewing teacher is not satisfied, he can give you another chance to rework your homework and to obtain more points.

In [2]:
import pandas as pd
import numpy as np

In [3]:
dat = pd.read_csv('MetObjects.csv')

c:\users\nicolas\appdata\local\programs\python\python37-32\lib\site-packages\IPython\core\interactiveshell.py:3018: DtypeWarning: Columns (7,8,9,10,11,18,27,28,29,30,31,32,33,34,35,36,37,39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Data consistency (Object Name, Title, Dimensions, Medium)

In [3]:
data = dat.copy()
diff = pd.DataFrame()
for col in data.columns:
    before = data[col].nunique()
    data[col] = data[col].astype(str).str.lower()
    after = data[col].nunique()
    if before != after:
        #print("Lowercasing {} : {} => {}".format(col,before,after))
        value = before - after
        df = pd.DataFrame([col,value]).T
        #display(df)
        diff = pd.concat([diff,df], ignore_index=True)
diff.columns = ['feature','diff']
display(diff.sort_values(diff.columns[1],ascending=False).head(4))
# Free some memory for later
del data
del diff

,feature,diff
2,Title,2047
19,Dimensions,1406
18,Medium,1308
1,Object Name,781


As you can see the 3 most inconsistent features are Title, Dimensions, Medium. We can now pick two of these features plus 'Object Name' feature as it is asked in the homework.

### Data integrity

In [44]:
# Checking that Object Begin Date is always inferior or equal to Object End Date
d = pd.DataFrame()
d = pd.concat([d,dat['Object Begin Date'],dat['Object End Date']], axis=1)

d['Object Date Integrity'] = np.where((d['Object Begin Date'] > d['Object End Date']), False, True)

# Displays invalid rows
# display(d.loc[d['Object Date Integrity'] == False])

countDF = d['Object Date Integrity'].groupby(d['Object Date Integrity']).agg({'count'})
display(countDF)

,count
Object Date Integrity,
False,198
True,472057


Above we checked the integrity between **Object Begin Date** and **Object End Date** features. 198 rows were marked as **False** in **Object Date Integrity**, showing that Object Begin Date is superior to Object End Date, which is not normal.  
Theses rows can be displayed by uncommenting the "*display*" line (line 8).  
Those rows should be imputed or checked to change the dates for the correct data.

### Data type conversion

In [5]:
# Convert <Metadata Date> column to datetime
dat['Metadata Date'] = dat['Metadata Date'].apply(pd.to_datetime)
# Convert <Object Begin Date> and <Object End Date> columns to numeric
dat[['Object Begin Date','Object End Date']] = dat[['Object Begin Date','Object End Date']].apply(pd.to_numeric)

We converted successfully above columns to datetime and numeric types. However, other columns were already imported as Pandas *"object"* type, which is already Categorical, so there's no need to convert it from object to object.

### Outliers Detection

In [6]:
# Artist End Date contains outliers (values like 9999)
d = dat['Artist End Date'].loc[dat['Artist End Date'].str.contains('9999', na=False)]
print(len(d.index))

10079


Artist End Date column contains some outliers containing 9999, which is anormal for a date. The code above counts these outliers.  
**We found 10079 outliers out of 472255 rows.**

### Missing Data Detection

In [7]:
missing = pd.DataFrame(dat.isnull().sum(),columns=['NaN count'])
display(missing.sort_values(missing.columns[0], ascending=False).head(40))

,NaN count
River,470157
State,469446
Locus,464930
County,463790
Artist Suffix,461509
Reign,461043
Locale,456700
Excavation,456292
Portfolio,450415
Subregion,450105


The Culture, Object Date and Artist Display Name columns contains missing values. We decide to convert them to a proper representation:
    <ul>
    <li>**Culture** : NaN to 'Unknown'</li>
    <li>**Object Date** : NaN to 'Unknown'</li>
    <li>**Artist Display Name** : NaN to 'Unknown Artist'</li>
    </ul>
The Credit Line column contains the least missing values so we can pick this feature as the one to impute rows.

In [8]:
dat['Culture'] = dat['Culture'].fillna('Unknown')
dat['Object Date'] = dat['Object Date'].fillna('Unknown')
dat['Artist Display Name'] = dat['Artist Display Name'].fillna('Unknown Artist')

dat.dropna(subset=['Credit Line'],inplace=True)